# Expressive Gestures

Laban/Bartenieff Movement Analysis (LMA) characterizes gestures as individual movements that convey meaning. In LMA, expressive gestures are defined as movements that convey emotions, intentions, or personal artistic expression, often characterized by their dynamic qualities, such as flow, weight, time, and space, which reflect the performer's inner state or interpretation of a theme. In this notebook, the same behavioral gestures will be analyzed to extract expressive qualities within these movements.

### Required Modules

In [ ]:
import cv2
import numpy as np
import os
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D

### Load Data and Define Paths

## Timing Variations

The aim of this section is to analyze the duration, rhythm, and tempo of the behavioral gestures to identify any variations that might indicate emotional expression. For example, a slower or faster pace in standing or sitting might reflect the performer's mood or intention. Harmonic waves will also be included for more context.

## Flow and Dynamics

The aim of this section is to examine the smoothness, continuity, and intensity of movements during the execution of behavioral gestures, as they may reveal the performer's emotional state. The following code calculates flow values for each frame in the behavioral gestures by measuring the Euclidean distance between the pose keypoints in consecutive frames, and then calculates the mean, standard deviation, minimum, and maximum flow values for each gesture to provide insights into the smoothness, continuity, and intensity of the movements.

## Body Posture and Alignment

The aim of this section is to assess the posture and alignment of the body during the performance of behavioral gestures. Differences in body posture can indicate the performer's emotions, intentions, or personal style. Here are the potential insights per metric:

* **Mean per relevant joint**: This data can be used to compare other gestures or to the ideal posture to see if there are any significant deviations.
* **Standard deviation per relevant joint**: High variability may indicate that the gesture is more difficult to execute or that the performer is expressing more emotion.
* **Calculated Range of Motion**: This data can be used to compare with other gestures or to the ideal posture to see if there are any significant differences.
* **Average Range of Motion**: Higher average range of motion may indicate that the performer is expressing more emotion or that the gesture is more dynamic.

## Sequantial Patterns

The aim of this section is to visualize the spatial patterns and range of motion during the performance of behavioral gestures. These are the potential insights:

* Identification of specific joint movements that are consistently used during a particular gesture
* Differences in joint movements and range of motion between different performers or performances
* Correlation between certain joint movements and emotional expression conveyed by the gesture
* Identification of unusual or unexpected joint movements that could indicate a unique style or interpretation of the gesture.